In [ ]:
# package import 

import os
import sys
import pandas as pd
pd.options.mode.copy_on_write = True 

from pathlib import Path
import cdata_utils
import numpy as np
import cdata_utils.utils
import datetime

from cdata_utils.utils import integerized

import json

#import cdata_utils.preprocess.read_and_clean_tabular
from cdata_utils.project_specific.psvd import (
    read_and_clean_PSVD_data__BL_consensus,
    categorize_PSVD_data,
    exclude_patients,
    reorder_some_categorical_values, 
    table1_psvd, 
    table1_psvd_spleen, 
    descriptive_df_from_masks, 
    masks_for_endpoint_1__decompensation, 
    masks_for_endpoint_2__death,
    make_y_delta,
    drop_non_numeric_columns,
    table_of_valid_entries
)

# from cdata_utils.descriptive.utils import (
#     get_occurencies,
#     #get_distinct_values_summary
# )


from cdata_utils.descriptive.basic_stats import (
    ordinal_numbers_frequency_str, 
    ordinal_numbers_frequency_ratio_str, 
    basic_stat_description,
    BasicStats,
    MEDIAN_IQR,
    MEAN_SD,
    N_PROCENT,
    ORDINAL_0_to_1,
    ORDINAL_1_to_2,
    ORDINAL_1_to_3,
    BasicStatsOrdinal,
    BasicStatsOrdinalFractions,
    ORDINAL_F_0_to_1,
    ORDINAL_F_1_to_2,
    ORDINAL_F_1_to_3,
    generate_descriptive_table, 
    generate_descriptive_table_no_index,
    describe
)

import cdata_utils.descriptive.basic_stats
from cdata_utils.descriptive.basic_stats import (
    basic_stat_description,
    BasicStatsOrdinal
    )

from cdata_utils.utils import (
    int_throw,
    masks_are_all_mutually_exclusive, 
    union_of_masks_is_complete,
    isdate_masks
)

# path info: 
if "cwatzenboeck" in os.getcwd(): # desktop 
    data_path = Path("/home/cwatzenboeck/Dropbox/work/data/livermodel/PSVD/")
    data_path_output=Path("/home/cwatzenboeck/data/psvd/output_coxph/")
else: # laptop 
    data_path = Path("/home/clemens/Dropbox/work/data/livermodel/PSVD/")
    # data_path = Path("/home/clemens/projects/project_liver_model/data/PSVD")
    
    
from cdata_utils.project_specific.psvd import (
    rename_columns_by_prefix, 
    read_renaming_dict,
    replace_,
    read_and_clean_PSVD_data__BL_consensus_NEW,
    load_EP1_EP2_data,
    relevant_column_names,
    relevant_column_names_clinical,
    categorize_PSVD_clinical_data,
    load_clinical_data,
)


    

In [ ]:
df1, df2 = load_EP1_EP2_data(data_path,  file_name = "data_PSVD_unified_3.xlsx")



In [ ]:


# df = read_and_clean_PSVD_data__BL_consensus_NEW(data_path=data_path, file_name = "data_PSVD_unified_3.xlsx", verbose=False, return_rename_dicts=False)

# describe(df).iloc[[0,8,9,10,11],:].transpose() # check that unique values match expectation


# describe(df).transpose().to_excel(data_path_output / "describe_baseline_data.xlsx")

In [ ]:
# df = categorize_PSVD_data(df, drop_modfied_colums=True)
# describe(df).iloc[8:,:].transpose() # check that unique values match expectation

In [ ]:
# debug
# df = read_and_clean_PSVD_data__BL_consensus_NEW(data_path=data_path, file_name = "data_PSVD_unified_3.xlsx", verbose=False, return_rename_dicts=False)
# # df, mapper_old2new, mapper_new2old = read_and_clean_PSVD_data__BL_consensus(data_path=data_path, file_name = "data_PSVD_unified_3.xlsx",   
# #                                                                             verbose=False, return_rename_dicts=True)
# df = reorder_some_categorical_values(df)

# df = categorize_PSVD_data(df, drop_modfied_colums=True)
    
# c1 = "1. Decompensation date"; event_column=c1
# df1 = make_y_delta(df, c1)
# df_ = df1[["BL CT Date", event_column, "event", "status"]]
# masks_date = isdate_masks(df_, event_column)
# df_[masks_date["zero"]]

In [ ]:
# read in data   # old data was "data_PSVD_unified_1.xlsx"

# df = read_and_clean_PSVD_data__BL_consensus_NEW(data_path=data_path, file_name = "data_PSVD_unified_3.xlsx", verbose=False, return_rename_dicts=False)
# # df, mapper_old2new, mapper_new2old = read_and_clean_PSVD_data__BL_consensus(data_path=data_path, file_name = "data_PSVD_unified_3.xlsx",   
# #                                                                             verbose=False, return_rename_dicts=True)
# df = reorder_some_categorical_values(df)

# df = categorize_PSVD_data(df, drop_modfied_colums=True)
    
c1 = "1. Decompensation date"
c2 = "Death"
# df1 = make_y_delta(df, c1)
# df2 = make_y_delta(df, c2)

# # drop negative event cases: 
# m = df1["event"] <= 0
# if sum(m)>0: 
#     print(f"Drop cases with negative time-to-event: ", list(df1[m]["ID"]), " for EP1")
# df1 = df1[~m]

# m = df2["event"] <= 0
# if sum(m)>0: 
#     print(f"Drop cases with negative time-to-event: ", list(df2[m]["ID"]), " for EP2")
# df2 = df2[~m]

# df1 = drop_non_numeric_columns(df1)
# df2 = drop_non_numeric_columns(df2)

df1, df2 = load_EP1_EP2_data(data_path,  file_name = "data_PSVD_unified_3.xlsx")


df1["event"] = df1["event"].apply(lambda x: float(x))
df1["status"] = df1["status"].apply(lambda x: int(x))

df2["event"] = df2["event"].apply(lambda x: float(x))
df2["status"] = df2["status"].apply(lambda x: int(x))

cc = ['BL Splanchnic thrombosis consensus binary cat. 1',
 'BL Splanchnic thrombosis consensus binary cat. 2',
 'BL Intrahepatic portal abnormalities consensus binary cat. 1',
 'BL Intrahepatic portal abnormalities consensus binary cat. 2',
 'BL Intrahepatic portal abnormalities consensus binary cat. 3',
 'BL Intrahepatic portal abnormalities consensus binary cat. 4',
 'BL Location consensus binary cat.']

for c in cc: 
    df1[c] = df1[c].apply(int)
    df2[c] = df2[c].apply(int)
    
df12 = df1.copy()
df12 = df12.rename(columns={"status": f"status EP={c1}",
                            "event":  f"event EP={c1}"})
df12[f"status EP={c2}"] = df2["status"]
df12[f"event EP={c2}"]  = df2["event"]



In [ ]:

describe(df12).transpose()

In [ ]:
describe(df1).transpose()


In [ ]:
# For a lot of the patients the last visit is missing ... therfore they need to be excluded for the EP2 part
# EP2 is however anyhow unlikely to work (to frew patients)
# df2[df2["event"].isnull()].filter(regex="Height|status|event|Death|Last Visit")
# table_of_valid_entries(df2)

In [ ]:
describe(df1)

In [ ]:
# describe(df1).to_excel(data_path / "output" / "EP1_descompensation_desciption.xlsx")
# describe(df2).to_excel(data_path / "output" / "EP2_death_desciption.xlsx")
# describe(df).to_excel(data_path / "output" / "All_desciption.xlsx")

In [ ]:
drop_negative_times_to_event_cases = True

## Clinical paramters:

In [ ]:
df1, df2 = load_EP1_EP2_data(data_path, file_name="data_PSVD_unified_3.xlsx", drop_negative_times_to_event_cases=True)

In [ ]:
df_c = load_clinical_data(data_path, file_name="data_PSVD_unified_3.xlsx", drop_modfied_colums=True)
# df1c = df1.join(df_c.drop(columns=['Sex (1=male, 2=female)']))
# df2c = df2.join(df_c.drop(columns=['Sex (1=male, 2=female)']))


# c1 = "1. Decompensation date"
# c2 = "Death"
# df1c = make_y_delta(df_c, c1)
# df2c = make_y_delta(df_c, c2)



# # drop negative event cases maybe: 
# m = df1c["event"] <= 0
# if drop_negative_times_to_event_cases and sum(m>0):
#     df1c = df1c[~m]
#     print(f"Drop cases with negative time-to-event: ", list(df1c[m]["ID"]), " for EP1")
# elif not drop_negative_times_to_event_cases and sum(m>0):
#     print(f"WARNING negative time-to-event: ", list(df1c[m]["ID"]), " for EP1 were NOT dropped")
    

# m = df2c["event"] <= 0
# if drop_negative_times_to_event_cases and sum(m>0):
#     df2c = df2c[~m]
#     print(f"Drop cases with negative time-to-event: ", list(df2c[m]["ID"]), " for EP2")
# elif not drop_negative_times_to_event_cases and sum(m>0):
#     print(f"WARNING negative time-to-event: ", list(df2c[m]["ID"]), " for EP2 were NOT dropped")


In [ ]:
df1.join(df_c.drop(columns=["Sex (1=male, 2=female)"]))